## Vidly 2.0 ASP.NET MVC REF.

## Architectural Pattern

Model - View - Controller ==> MVC

### Model:
- Application Data and behaviour in terms of its problem domain
- Independent of the UI
### Domain Model of Vidly 2.0
    - Movie
    - Customer 
    - Rental 
    - Transaction
### View:
- The HTML markup that we display to the user
### Controller
- Responsible for handaling an HTTP Request
- vidly.-com/movies
- takes data from model puts it in view
### Router 
- Selects the right controller
- decides which controller will use


### App_Start
#### RouteConfig
**routes.MapRoute(<br>
                name: "Default",<br>
                url: "{controller}/{action}/{id}",<br>
                defaults: new { controller = "Home", action = "Index", id = UrlParameter.Optional }<br>**
#### Details                
- the default link is set to home
- /movies/popular
    - movies => MoviesController
        - popular => Popular(); popular is a method in MoviesController
- /movies/edit/1
    - will call Edit(int id) from MoviesController
- /movies ; action name is not specified
    - by default: MoviesController().Index()
- id is optional parameter

### Solution Explorer
- model : contains domain class
- controller
    -AccountController.cs
    -HomeController.cs
    -ManageController.cs
- views ; folders named after controller
    -Account
    -Home
    -Manage
    -shared ; across different controller
- packages.config
    -dependencies external
    

## First Step

- create a domain class in model
- name: Movie.cs
    - adding property: public int Id {get; set;}
    - property for representing state
- create a controller
- name : MoviesController : Controller
    - action : public ActionResult Random()
        - var movie = new Movie() {Name = "Shrek!"}
        - return View(movie); have to create a view in Views
- view > Movie > add new view
    - dialog box: Random add _layout
        - first line : @model vidly.Models.Movie
        - in <*h2>@Model.Name</h2*>

## Changing Theme
- download bootstrap theme download bootsrap.css file
- add it in the content folder 
- app_start > bundleconfig
    -last bundle is css assets: change accordingly
    - ctrl + f5 to build and run

# Fundamentals

### Action Result
    - ActionResult Types
        - View() PartitalView() Content() RedirectToAction()
    - Action Parameters
        - Request > MVC framework > Action
        - sources : 
            - URL : /movies/edit/1 
            - In Query string : /movies/edit?id=1 ;default route is id, value will be id untill routeconfig is changed
            - In the form of data: id=1
            -*public ActionResult Index(int? pageIn, string sB)*
                -here pageIn is nullable
                
### Custom Route
    - RouteConfig ; target : /movies/released/2015/04
        - before default route bcz oder of these matters
        - most specific first then most generic
        - *routes.MapRoute( 
                    Name: "MoviesByReleaseDate",
                    url: "movies/released/{year}/{month}", 
                    default: new {controller = "Movies", action = 
                                               "ByReleasedDate"}
                    new {year = @"\d{4}",month = @"\d{2}"});
                    ^^ is a constraint can use 2015|2016
                                               
    - route created
    - create ActionResult for customRoute
    - MoviesController :
        - * public ActionResult ByReleaseDate(int year, int month)
                        {return Content(year + "/" + month)}
                        
                        
### Custom Route Cleaner Approach : Attribute Routing
    - RouteConfig.cs
        - *routes.MapMvcAttributeRoutes()
        
    - MoviesController.cs
        - before ByReleaseDate class
        - [Route("movies/released/{year}/{month:regex(\\d{4})
                                    :range(1,12)}")]
                                    
        - google : ASP.NET MVC Attribute Route Constraints
        
### Passing Data To Views
#### approach #1
    - MoviesController.cs
        - *ViewData["Movie"] = movie; 
            return view();
    - Views > Movies > Random.cshtml
        - <h2>@(((Movie)ViewData["Movie"]).Name</h2>
#### approach #2
    - MoviesController.cs
        - ViewBag.Movie = movie;
    - Views > Movies > Random.cshtml
        - <h2>@ViewBag.Movie/h2>
### View Models

- in random view we don't see any customer 
- no relation between domains
- view model : a model specificly made for view
    - customer.cs in model
        - * public int Id {get; set;}
            public string Name {get; set;}
    - create a folder: ViewModel in SolutionExplorer
        - add a new class : RandomMovieViewModel
            - * public Movie Movie {get; set;}
                public List<*Customer> Customers {get; set;}
    
    - MoviesController:
        - *var customers = new List<*Customer>
            {new Customer {Name = "Customer 1"}}
        - var viewModel = new RandomMovieViewModel
            {Movie = movie, Customers = Customers};
            
    - change random.cshtml
        - @model Vidly.ViewModels.RandomMovieViewModel
        - <ul>
                * foreach (var customer in Model.customers)
                {
                    <li>@customer.Name</li>
                }
          </ul>
### Partial Views
    - _Layout.cshtml
        - div > navbar
        - create new view : > _NavBar *partial view check
        - cut div navbar from _layout paste in _NavBar
        - in place of div navbar: @Html.Partial("_NavBar") 

            


## Working with Data
### Entity Framework
    - Object Relational Mapper
    - LinQ to query Dbset
    - workflow
        - Code first
        - Database firt
### Code First

- tools > NuGet PM > Package Manager Controll
- enable-migration ; to enable it.
- add-migration InitialModel ; under migration folder new file created
- models> identityModels.cs
    - create new prop *public DbSet<Customer*> Customers {g;s;}
- PM>add-migration initalModel -force (overwrite)
- new table customer created
- PM > Update-database 
- app_data > database file > check customer table

### Changing the Model
- Customer Model> IsSubscribedToNewsLetter, MembershipType
- Aim for small migrations
- model> customer.cs
    - * public bool IsSubscribedToNewsLetter {g;s;}
- PM > add-migration AddIsSubscribedToCustomer
- PM > Update-database
- Model create MembershipType.cs
    - * public byte Id {}  ;every entity must have a key ID
    - * public short SignUp fee {}
    - * public byte DurationInMonth {}
    - * public byte DiscounRate {}
- Model Customer.cs
    - * public Membershiptype Membershiptype {}
    - * public byte MembershipTypeId {} ; foreign id
- PM > add-migration AddMembershiptype
- PM > Update-database



### Seeding the Database

- PM > add-migration PopulateMembershipType
- Migration > populateMembershipType
    - Inside public override void Up()
        - * Sql("INSERT INTO MembershipTypes (Id, SignUpFee, DurationInMonths, DiscountRate) VALUES (1, 0, 0, 0)");
            Sql("INSERT INTO MembershipTypes (Id, SignUpFee, DurationInMonths, DiscountRate) VALUES (2, 30, 1, 10)");
            Sql("INSERT INTO MembershipTypes (Id, SignUpFee, DurationInMonths, DiscountRate) VALUES (3, 90, 3, 15)");
            Sql("INSERT INTO MembershipTypes (Id, SignUpFee, DurationInMonths, DiscountRate) VALUES (4, 300, 12, 20)");
- PM > Update-database



### Overriding Convenvention

- Models > Customer.cs ; using data annotation
- [Required] 
  [StringLength(225)] before name property
- PM > add-migration ApplyAnnotationToCustomerName
- PM > Update-database



### Querying Object

- ControllerCustomer.cs 
    - *private ApplicationDbContext _context
    - *public CustomerController()
        {_context = new ApplicationDbContext();}
    - *override Dispose : _context.Dispose();
    - *public ViewResult Index()
        { var customers = _context.customers.ToList()
        return View(customers);}
    - GetCustomers() => _context.customers.ToList()


### Eager Loading

- views > customer > Index.cshtml
    - add heading Discount Rate
    - add another td> @customer.membershiptype.discount%
    - Error
- CustomersControllers.cs
    - var customers = _context.*Include(c =>                                    c.membershipTYoe)*customers.ToList()
    - using system.data.entity
    